In [10]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from transformers import AutoModel, AutoTokenizer
import re
import gensim
import torch

In [35]:
class SentenceEmbeddingsDemo:
    def __init__(self):
        # Load pre-trained Sentence-BERT model
        self.model = SentenceTransformer('all-MiniLM-L6-v2')

        # Example sentences
        self.sentences = [
            "Fotosintesis adalah proses tumbuhan hijau mengubah energi cahaya menjadi energi kimia dalam bentuk glukosa", #Tes 1
            "Tumbuhan hijau melakukan fotosintesis untuk menghasilkan glukosa dengan memanfaatkan energi dari cahaya matahari", #Tes 2
            "Fotosintesis adalah bash terminal yang digunakan di mata kuliah sistem operasi untuk menghasilkan glukosa", #Kalimat ga nyambung tapi sebagian konteksnya ada di teks
            "Fotosintesis hijau energi kimia glukosa", #Kalimat ga nyambung, tapi semua konteksnya ada di teks
            "Kalau kata temen saya si Khansa, sepuh ML, fotosintesis itu yang bisa jadiin energi kimia dari cahaya dah, terus ngehasilin glukosa", #Kalimat banyak bacot tapi masi nyambung
            "Menurut saya, IPB adalah kampus terbaik", #Kalimat sampah
            "Fotosintesis tuh semacam proses alam dari taneman, dimana dia ngeconvert resource yang ada di alam jadi makanan yang bisa dipakai oleh kita" #Kalimat yang maknanya sama, tapi beda secara tekstual
        ]

    def demonstrate_sentence_embeddings(self):
        """
        Show capabilities of sentence embeddings
        """
        # Generate embeddings
        embeddings = self.model.encode(self.sentences)

        # Compute semantic similarity
        print("Semantic Similarity Comparison:")
        for i in range(len(self.sentences)):
            for j in range(i+1, len(self.sentences)):
                similarity = np.dot(embeddings[i], embeddings[j]) / (
                    np.linalg.norm(embeddings[i]) * np.linalg.norm(embeddings[j])
                )
                print(f"Similarity between:\n'{self.sentences[i]}'\nand\n'{self.sentences[j]}': {similarity:.4f}\n")

In [36]:
class TransformerEmbeddingsDemo:
    def __init__(self):
        # Load pre-trained BERT model
        model_name = 'bert-base-uncased'
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def mean_pooling(self, model_output, attention_mask):
        """
        Mean pooling to get sentence embedding
        """
        token_embeddings = model_output[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    def demonstrate_transformer_embeddings(self):
        """
        Show capabilities of transformer-based embeddings
        """
        sentences = [
            "Fotosintesis adalah proses tumbuhan hijau mengubah energi cahaya menjadi energi kimia dalam bentuk glukosa", #Tes 1
            "Tumbuhan hijau melakukan fotosintesis untuk menghasilkan glukosa dengan memanfaatkan energi dari cahaya matahari", #Tes 2
            "Fotosintesis adalah bash terminal yang digunakan di mata kuliah sistem operasi untuk menghasilkan glukosa", #Kalimat ga nyambung tapi sebagian konteksnya ada di teks
            "Fotosintesis hijau energi kimia glukosa", #Kalimat ga nyambung, tapi semua konteksnya ada di teks
            "Kalau kata temen saya si Khansa, sepuh ML, fotosintesis itu yang bisa jadiin energi kimia dari cahaya dah, terus ngehasilin glukosa", #Kalimat banyak bacot tapi masi nyambung
            "Menurut saya, IPB adalah kampus terbaik", #Kalimat sampah
            "Fotosintesis tuh semacam proses alam dari taneman, dimana dia ngeconvert resource yang ada di alam jadi makanan yang bisa dipakai oleh kita" #Kalimat yang maknanya sama, tapi beda secara tekstual
        ]

        # Tokenize sentences
        encoded_input = self.tokenizer(
            sentences,
            padding=True,
            truncation=True,
            return_tensors='pt'
        )

        # Generate embeddings
        with torch.no_grad():
            model_output = self.model(**encoded_input)

        # Apply mean pooling
        sentence_embeddings = self.mean_pooling(model_output, encoded_input['attention_mask'])

        print("Transformer Embedding Characteristics:")
        print("Embedding Dimensions:", sentence_embeddings.shape)

        # Compute cosine similarities
        print("\nSemantic Similarity:")
        for i in range(len(sentences)):
            for j in range(i+1, len(sentences)):
                similarity = torch.nn.functional.cosine_similarity(
                    sentence_embeddings[i].unsqueeze(0),
                    sentence_embeddings[j].unsqueeze(0)
                ).item()
                print(f"Similarity between:\n'{sentences[i]}'\nand\n'{sentences[j]}': {similarity:.4f}\n")


In [37]:
# Demonstration function
def main():
    print("\n--- Sentence Embeddings ---")
    sentence_demo = SentenceEmbeddingsDemo()
    sentence_demo.demonstrate_sentence_embeddings()

    print("\n--- Transformer Embeddings ---")
    transformer_demo = TransformerEmbeddingsDemo()
    transformer_demo.demonstrate_transformer_embeddings()

if __name__ == "__main__":
    main()


--- Sentence Embeddings ---
Semantic Similarity Comparison:
Similarity between:
'Fotosintesis adalah proses tumbuhan hijau mengubah energi cahaya menjadi energi kimia dalam bentuk glukosa'
and
'Tumbuhan hijau melakukan fotosintesis untuk menghasilkan glukosa dengan memanfaatkan energi dari cahaya matahari': 0.8942

Similarity between:
'Fotosintesis adalah proses tumbuhan hijau mengubah energi cahaya menjadi energi kimia dalam bentuk glukosa'
and
'Fotosintesis adalah bash terminal yang digunakan di mata kuliah sistem operasi untuk menghasilkan glukosa': 0.8207

Similarity between:
'Fotosintesis adalah proses tumbuhan hijau mengubah energi cahaya menjadi energi kimia dalam bentuk glukosa'
and
'Fotosintesis hijau energi kimia glukosa': 0.8960

Similarity between:
'Fotosintesis adalah proses tumbuhan hijau mengubah energi cahaya menjadi energi kimia dalam bentuk glukosa'
and
'Kalau kata temen saya si Khansa, sepuh ML, fotosintesis itu yang bisa jadiin energi kimia dari cahaya dah, terus n